## Initial setup

Picks GPU, prepares file structure and initializes variables

In [1]:
import tensorflow as tf
import os
import sys

physical_devices = tf.config.list_physical_devices('GPU')
print("Avaliable GPUs:")
for device in physical_devices:
    print(device.name)
print(f"Chosen GPU: {physical_devices[-1].name}")
tf.config.set_visible_devices(physical_devices[-1], 'GPU')

path_common= os.path.dirname(os.path.abspath('')) + "/common/"
sys.path.append(path_common)
print("Current working directory:", os.getcwd())

model, hpo_search_function, hpo_n_epochs_max, hpo_max_trials, hpo_batch_size = [None] * 5

2024-03-25 15:24:35.476100: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-25 15:24:35.496302: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 15:24:35.496314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 15:24:35.496848: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-25 15:24:35.500821: I tensorflow/core/platform/cpu_feature_guar

Avaliable GPUs:
/physical_device:GPU:0
Chosen GPU: /physical_device:GPU:0
Current working directory: /home/slagarp/Master-Thesis/src/main


2024-03-25 15:24:36.283658: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-25 15:24:36.295927: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-25 15:24:36.295962: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


# Pick your model

Available options:

- FCN:                Fully Convultional Neural Network
- ResNet:             Residual Neural Network
- LSTM_FCN:           Long Short Term Memory with Fully Convulutional Network
- Transformer:        ransformer Based model

In [2]:
from ModelFuncs import FCN, ResNet, LSTM_FCN, Transformer
model = FCN()

## Hyper Parameter Optimization

Pick search function and tuning parameters

Available search functions:
- random:               Random Search
- bayesian:             Bayesian Optimization

Available parameters:
- hpo_n_epochs_max
- hpo_max_trials
- hpo_batch_size

In [3]:
from HyperParameterTuning import Tuner

hpo_search_function = "bayesian"

hpo_n_epochs_max = 3
hpo_max_trials = 5
hpo_batch_size = 128

hpo_tuner = Tuner(model, n_epochs_max = hpo_n_epochs_max, max_trials = hpo_max_trials, batch_size = hpo_batch_size, search_function=hpo_search_function, skip_folds = 3)

Run hyper parameter tuning

In [4]:
#%%capture output
hpo_tuner.tune()
#output.show()

Trial 5 Complete [00h 00m 42s]
val_loss: 1.8514328598976135

Best val_loss So Far: 1.8468377590179443
Total elapsed time: 00h 03m 34s
Results summary
Results in ../../results/bayesian/FCN
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 3 summary
Hyperparameters:
filters_block_0: 32
kernel_size_block_0: 9
Score: 1.8468377590179443

Trial 1 summary
Hyperparameters:
filters_block_0: 32
kernel_size_block_0: 13
Score: 1.8511515855789185

Trial 4 summary
Hyperparameters:
filters_block_0: 32
kernel_size_block_0: 7
Score: 1.8514328598976135

Trial 2 summary
Hyperparameters:
filters_block_0: 32
kernel_size_block_0: 3
Score: 1.85228031873703

Trial 0 summary
Hyperparameters:
filters_block_0: 32
kernel_size_block_0: 11
Score: 1.8541500568389893


In [5]:
hpo_tuner.summary()

Reloading Tuner from ../../results/bayesian/FCN/tuner0.json
Results summary
Results in ../../results/bayesian/FCN
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 3 summary
Hyperparameters:
filters_block_0: 32
kernel_size_block_0: 9
Score: 1.8468377590179443

Trial 1 summary
Hyperparameters:
filters_block_0: 32
kernel_size_block_0: 13
Score: 1.8511515855789185

Trial 4 summary
Hyperparameters:
filters_block_0: 32
kernel_size_block_0: 7
Score: 1.8514328598976135

Trial 2 summary
Hyperparameters:
filters_block_0: 32
kernel_size_block_0: 3
Score: 1.85228031873703

Trial 0 summary
Hyperparameters:
filters_block_0: 32
kernel_size_block_0: 11
Score: 1.8541500568389893


## Train Model

Get best hyper parameters

In [6]:
tuner_object = hpo_tuner.get_tuner()

tuner_object.reload()

best_hps = tuner_object.get_best_hyperparameters()[0]

best_model = model.build(best_hps)

best_model.summary()



Reloading Tuner from ../../results/bayesian/FCN/tuner0.json
Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, None, 1)]         0         
                                                                 
 conv1d_11 (Conv1D)          (None, None, 32)          320       
                                                                 
 batch_normalization_11 (Ba  (None, None, 32)          128       
 tchNormalization)                                               
                                                                 
 activation_11 (Activation)  (None, None, 32)          0         
                                                                 
 global_average_pooling1d_1  (None, 32)                0         
 1 (GlobalAveragePooling1D)                                      
                                                                

Train model

In [7]:
from ModelTrainer import ModelTrainer
mt=ModelTrainer(n_epochs_max=10, tb_folder="../../results/tb_logs/standard/"); 
mt.start_train_es(best_model)

=== Epoch: 0 ===
397/397 [==============================] - 3s 8ms/step - loss: 1.8129 - accuracy: 0.3060 - val_loss: 1.7814 - val_accuracy: 0.3051
  prep time: 1.4 sec   train time: 3.5 sec
=== Epoch: 1 ===
397/397 [==============================] - 3s 7ms/step - loss: 1.7415 - accuracy: 0.3472 - val_loss: 1.6877 - val_accuracy: 0.3803
  prep time: 1.5 sec   train time: 3.0 sec
=== Epoch: 2 ===
397/397 [==============================] - 3s 7ms/step - loss: 1.7225 - accuracy: 0.3522 - val_loss: 1.6718 - val_accuracy: 0.3836
  prep time: 1.4 sec   train time: 3.0 sec
=== Epoch: 3 ===
397/397 [==============================] - 3s 7ms/step - loss: 1.7156 - accuracy: 0.3532 - val_loss: 1.6661 - val_accuracy: 0.3916
  prep time: 1.4 sec   train time: 3.0 sec
=== Epoch: 4 ===
397/397 [==============================] - 3s 7ms/step - loss: 1.7136 - accuracy: 0.3541 - val_loss: 1.6586 - val_accuracy: 0.3925
  prep time: 1.4 sec   train time: 3.0 sec
=== Epoch: 5 ===
397/397 [===================

(0.3878105580806732,
 1.6439214944839478,
 0.3901747763156891,
 1.6437766551971436,
 0.4890746772289276,
 1.6773719787597656,
 9)